#### find most frequent 9-mers with mismatches and reverse compelemtns in the region suggested by the minimum skew as ori
get minimum skew, try a small window either starting, ending, or centered at the position of minimum skew.

In [48]:
from Bio import SeqIO

Step 0: import data

In [49]:
fasta_in = '/Users/npajek/Documents/fri_biolab/coursera/bioinformatics-specialization/01_finding_hidden_messages_in_DNA/data/Salmonella_enterica.txt'

for record in SeqIO.parse(fasta_in, 'fasta'):
    sequence = str(record.seq)

step 1: Get minimum skew index

In [50]:
def min_skew_idx(sequence):
    skew = 0
    skew_list = [0] + [skew := skew + (1 if nt == 'G' else -1 if nt == 'C' else 0) for nt in sequence]
    min_value = min(skew_list)
    min_indices = [i for i, x in enumerate(skew_list) if x == min_value]
    return min_indices

# Test the function
min_skew_indices = min_skew_idx(sequence)
print("Indices with minimal skew:", *min_skew_indices, sep=' ')

Indices with minimal skew: 3764856 3764858


step2: select window

In [51]:
region = sequence[min_skew_indices[0]:min_skew_indices[0]+500]

step 3: identify most frequent 9-mers with 1 mismatch and reverse complements within this window

In [52]:
def frequent_words(sequence, k):
    kmer_freq = {}

    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        kmer_freq[kmer] = kmer_freq.get(kmer, 0) + 1
    return kmer_freq

# Define funtion find_clumps where k = kmer len, L = window len and t is number of occurrences
def find_clumps(sequence, k, L, t):
    patterns = []
    for i in range(0, len(sequence)-L):
        window = sequence[i: i+L]
        freq_map = frequent_words(window, k)
        patterns.extend([key for key, val in freq_map.items() if val >= t])
    return set(patterns)

# define hamming_distance
def hamming_distance(p, q):
    return sum(1 for x, y in zip(p, q) if x != y)

# define reverse complement
def reverse_complement(sequence):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    rc_seq = "".join(complement.get(base, base) for base in reversed(sequence))
    return rc_seq

def neighbors(pattern, d):
    nucleotides = ['A', 'C', 'G', 'T']
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return set(nucleotides)
    neighborhood = set()
    suffix_neighbors = neighbors(pattern[1:], d) #recursive call
    for text in suffix_neighbors:
        if hamming_distance(pattern[1:], text) < d:
            for x in nucleotides:
                neighborhood.add(x + text)
        else:
            neighborhood.add(pattern[0] + text)
    return list(neighborhood)

def frequent_words_with_mismatches(seq, k, d):
    freq_map = {}
    for i in range(0, len(seq)-k+1):
        pattern = seq[i:i+k]
        neighborhood = neighbors(pattern, d)
        for j in range(0,len(neighborhood)-1):
            neighbor = neighborhood[j]
            freq_map[neighbor] = freq_map.get(neighbor, 0) + 1
    for i in range(0, len(seq)-k+1):
        pattern = reverse_complement(seq[i:i+k])
        neighborhood = neighbors(pattern, d)
        for j in range(0,len(neighborhood)-1):
            neighbor = neighborhood[j]
            freq_map[neighbor] = freq_map.get(neighbor, 0) + 1
    m = max(freq_map.values())
    patterns = [kmer for kmer, freq in freq_map.items() if freq == m]
    return patterns

In [53]:
k = 9
d = 1

frequent_words_with_mismatches(region, k, d)

['CCCGGATCC', 'CGGATCCGG', 'GGATCCGGG', 'GGATCCTGG', 'CCAGGATCC', 'CCGGATCCG']